# Digital Asset Portfolio Optimization & Backtesting Analysis

## Objective
Demonstrate how strategic alt exposure (even large-cap alts) can potentially generate superior risk-adjusted returns compared to a pure Bitcoin buy-and-hold strategy.

## Portfolios Under Analysis
1. **Benchmark**: 100% Bitcoin (BTC)
2. **3-Asset Portfolio**: BTC + ETH + SOL (optimized weights)
3. **5-Asset Portfolio**: BTC + ETH + SOL + BNB + XRP (optimized weights)

## Definition of "Best"

We define "best" using multiple criteria to provide a holistic view:

### Primary Metric: Sharpe Ratio (Risk-Adjusted Returns)
- **Why**: Investors care about returns relative to risk taken. A 200% return with 150% volatility is worse than 100% return with 40% volatility from a risk-adjusted perspective.
- **Formula**: (Portfolio Return - Risk-Free Rate) / Portfolio Volatility

### Secondary Metrics:
- **CAGR**: Compound Annual Growth Rate - raw performance
- **Max Drawdown**: Worst peak-to-trough decline - downside risk
- **Sortino Ratio**: Like Sharpe but only penalizes downside volatility
- **Calmar Ratio**: CAGR / Max Drawdown - return per unit of drawdown risk

---

In [1]:
# Install required packages if needed
# !pip install yfinance pandas numpy matplotlib seaborn scipy

In [2]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
from scipy.optimize import minimize
from itertools import product
import warnings
warnings.filterwarnings('ignore')

# Set style
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette('husl')

# Constants
RISK_FREE_RATE = 0.03  # 3% annual risk-free rate
TRADING_DAYS = 365     # Crypto trades 365 days/year
INITIAL_INVESTMENT = 10000

print("Libraries loaded successfully!")

Libraries loaded successfully!


## 1. Data Acquisition

Fetch historical price data from Yahoo Finance. We'll use the longest common history where all assets have data.

In [3]:
# Define assets
ASSETS = {
    'BTC': 'BTC-USD',
    'ETH': 'ETH-USD', 
    'SOL': 'SOL-USD',
    'BNB': 'BNB-USD',
    'XRP': 'XRP-USD'
}

# Fetch data - SOL launched in March 2020, so we start from April 2020
START_DATE = '2020-04-01'
END_DATE = datetime.now().strftime('%Y-%m-%d')

print(f"Fetching data from {START_DATE} to {END_DATE}...")

# Download all assets
prices = pd.DataFrame()
for name, ticker in ASSETS.items():
    data = yf.download(ticker, start=START_DATE, end=END_DATE, progress=False)
    prices[name] = data['Adj Close']
    print(f"  {name}: {len(data)} days of data")

# Drop any rows with missing data
prices = prices.dropna()
print(f"\nTotal aligned data points: {len(prices)} days")
print(f"Date range: {prices.index[0].strftime('%Y-%m-%d')} to {prices.index[-1].strftime('%Y-%m-%d')}")

Fetching data from 2020-04-01 to 2026-01-20...
YF.download() has changed argument auto_adjust default to True


KeyError: 'Adj Close'

In [ ]:
# Calculate daily returns
returns = prices.pct_change().dropna()

# Quick sanity check
print("Daily Returns Summary Statistics:")
print(returns.describe().round(4))

## 2. Portfolio Metrics Functions

Define functions to calculate all key portfolio metrics.

In [ ]:
def calculate_portfolio_returns(weights, returns_df):
    """
    Calculate portfolio daily returns given weights and asset returns.
    """
    return (returns_df * weights).sum(axis=1)


def calculate_portfolio_value(weights, prices_df, initial_investment=INITIAL_INVESTMENT):
    """
    Calculate portfolio value over time with buy-and-hold (no rebalancing).
    """
    # Initial allocation
    initial_prices = prices_df.iloc[0]
    units = (initial_investment * np.array(weights)) / initial_prices
    
    # Portfolio value over time
    portfolio_values = (prices_df * units).sum(axis=1)
    return portfolio_values


def calculate_metrics(portfolio_values, daily_returns=None):
    """
    Calculate comprehensive portfolio metrics.
    """
    if daily_returns is None:
        daily_returns = portfolio_values.pct_change().dropna()
    
    # Basic metrics
    total_return = (portfolio_values.iloc[-1] / portfolio_values.iloc[0]) - 1
    
    # Time period in years
    years = (portfolio_values.index[-1] - portfolio_values.index[0]).days / 365
    
    # CAGR
    cagr = (portfolio_values.iloc[-1] / portfolio_values.iloc[0]) ** (1/years) - 1
    
    # Volatility (annualized)
    volatility = daily_returns.std() * np.sqrt(TRADING_DAYS)
    
    # Sharpe Ratio
    sharpe = (cagr - RISK_FREE_RATE) / volatility if volatility != 0 else 0
    
    # Max Drawdown
    rolling_max = portfolio_values.expanding().max()
    drawdowns = (portfolio_values - rolling_max) / rolling_max
    max_drawdown = drawdowns.min()
    
    # Sortino Ratio (only downside volatility)
    downside_returns = daily_returns[daily_returns < 0]
    downside_std = downside_returns.std() * np.sqrt(TRADING_DAYS)
    sortino = (cagr - RISK_FREE_RATE) / downside_std if downside_std != 0 else 0
    
    # Calmar Ratio
    calmar = cagr / abs(max_drawdown) if max_drawdown != 0 else 0
    
    # Best/Worst Days
    best_day = daily_returns.max()
    worst_day = daily_returns.min()
    
    # Win Rate
    win_rate = (daily_returns > 0).sum() / len(daily_returns)
    
    return {
        'total_return': total_return,
        'cagr': cagr,
        'volatility': volatility,
        'sharpe': sharpe,
        'sortino': sortino,
        'max_drawdown': max_drawdown,
        'calmar': calmar,
        'best_day': best_day,
        'worst_day': worst_day,
        'win_rate': win_rate,
        'final_value': portfolio_values.iloc[-1]
    }


print("Metric functions defined.")

## 3. Portfolio Optimization

Find optimal weights for 3-asset and 5-asset portfolios using grid search to maximize Sharpe Ratio.

In [ ]:
def optimize_portfolio_grid(assets_list, returns_df, prices_df, step=5):
    """
    Grid search optimization to find weights that maximize Sharpe ratio.
    Step size in percentage points (e.g., 5 = 0%, 5%, 10%, ..., 100%)
    """
    n_assets = len(assets_list)
    asset_returns = returns_df[assets_list]
    asset_prices = prices_df[assets_list]
    
    best_sharpe = -np.inf
    best_weights = None
    best_metrics = None
    
    # Generate all weight combinations that sum to 100
    weight_range = range(0, 101, step)
    
    results = []
    
    if n_assets == 3:
        for w1 in weight_range:
            for w2 in weight_range:
                w3 = 100 - w1 - w2
                if 0 <= w3 <= 100:
                    weights = np.array([w1, w2, w3]) / 100
                    pv = calculate_portfolio_value(weights, asset_prices)
                    metrics = calculate_metrics(pv)
                    results.append({
                        'weights': weights,
                        **{f'w_{a}': w for a, w in zip(assets_list, weights)},
                        **metrics
                    })
                    if metrics['sharpe'] > best_sharpe:
                        best_sharpe = metrics['sharpe']
                        best_weights = weights
                        best_metrics = metrics
                        
    elif n_assets == 5:
        for w1 in weight_range:
            for w2 in weight_range:
                for w3 in weight_range:
                    for w4 in weight_range:
                        w5 = 100 - w1 - w2 - w3 - w4
                        if 0 <= w5 <= 100:
                            weights = np.array([w1, w2, w3, w4, w5]) / 100
                            pv = calculate_portfolio_value(weights, asset_prices)
                            metrics = calculate_metrics(pv)
                            results.append({
                                'weights': weights,
                                **{f'w_{a}': w for a, w in zip(assets_list, weights)},
                                **metrics
                            })
                            if metrics['sharpe'] > best_sharpe:
                                best_sharpe = metrics['sharpe']
                                best_weights = weights
                                best_metrics = metrics
    
    return best_weights, best_metrics, pd.DataFrame(results)


print("Optimization function defined.")

In [ ]:
# Optimize 3-Asset Portfolio (BTC, ETH, SOL)
print("Optimizing 3-Asset Portfolio (BTC, ETH, SOL)...")
assets_3 = ['BTC', 'ETH', 'SOL']
optimal_weights_3, optimal_metrics_3, results_3 = optimize_portfolio_grid(assets_3, returns, prices, step=5)

print(f"\nOptimal 3-Asset Weights:")
for asset, weight in zip(assets_3, optimal_weights_3):
    print(f"  {asset}: {weight*100:.1f}%")
print(f"Sharpe Ratio: {optimal_metrics_3['sharpe']:.3f}")

In [ ]:
# Optimize 5-Asset Portfolio (BTC, ETH, SOL, BNB, XRP)
print("Optimizing 5-Asset Portfolio (BTC, ETH, SOL, BNB, XRP)...")
print("This may take a moment due to larger search space...")
assets_5 = ['BTC', 'ETH', 'SOL', 'BNB', 'XRP']
optimal_weights_5, optimal_metrics_5, results_5 = optimize_portfolio_grid(assets_5, returns, prices, step=10)

print(f"\nOptimal 5-Asset Weights:")
for asset, weight in zip(assets_5, optimal_weights_5):
    print(f"  {asset}: {weight*100:.1f}%")
print(f"Sharpe Ratio: {optimal_metrics_5['sharpe']:.3f}")

In [ ]:
# Calculate Bitcoin-only benchmark
print("Calculating Bitcoin benchmark...")
btc_values = calculate_portfolio_value([1.0], prices[['BTC']])
btc_metrics = calculate_metrics(btc_values)

print(f"\nBitcoin (100% BTC) Performance:")
print(f"  Total Return: {btc_metrics['total_return']*100:.1f}%")
print(f"  Sharpe Ratio: {btc_metrics['sharpe']:.3f}")

## 4. Portfolio Comparison

Compare all three portfolios across key metrics.

In [ ]:
# Calculate portfolio values for visualization
portfolio_3_values = calculate_portfolio_value(optimal_weights_3, prices[assets_3])
portfolio_5_values = calculate_portfolio_value(optimal_weights_5, prices[assets_5])

# Create comparison dataframe
comparison_data = {
    'Metric': [
        'Total Return',
        'CAGR',
        'Volatility (Annualized)',
        'Sharpe Ratio',
        'Sortino Ratio', 
        'Max Drawdown',
        'Calmar Ratio',
        'Best Day',
        'Worst Day',
        'Win Rate',
        f'Final Value (${INITIAL_INVESTMENT:,} invested)'
    ],
    '100% BTC': [
        f"{btc_metrics['total_return']*100:.1f}%",
        f"{btc_metrics['cagr']*100:.1f}%",
        f"{btc_metrics['volatility']*100:.1f}%",
        f"{btc_metrics['sharpe']:.3f}",
        f"{btc_metrics['sortino']:.3f}",
        f"{btc_metrics['max_drawdown']*100:.1f}%",
        f"{btc_metrics['calmar']:.3f}",
        f"{btc_metrics['best_day']*100:.1f}%",
        f"{btc_metrics['worst_day']*100:.1f}%",
        f"{btc_metrics['win_rate']*100:.1f}%",
        f"${btc_metrics['final_value']:,.0f}"
    ],
    '3-Asset Optimal': [
        f"{optimal_metrics_3['total_return']*100:.1f}%",
        f"{optimal_metrics_3['cagr']*100:.1f}%",
        f"{optimal_metrics_3['volatility']*100:.1f}%",
        f"{optimal_metrics_3['sharpe']:.3f}",
        f"{optimal_metrics_3['sortino']:.3f}",
        f"{optimal_metrics_3['max_drawdown']*100:.1f}%",
        f"{optimal_metrics_3['calmar']:.3f}",
        f"{optimal_metrics_3['best_day']*100:.1f}%",
        f"{optimal_metrics_3['worst_day']*100:.1f}%",
        f"{optimal_metrics_3['win_rate']*100:.1f}%",
        f"${optimal_metrics_3['final_value']:,.0f}"
    ],
    '5-Asset Optimal': [
        f"{optimal_metrics_5['total_return']*100:.1f}%",
        f"{optimal_metrics_5['cagr']*100:.1f}%",
        f"{optimal_metrics_5['volatility']*100:.1f}%",
        f"{optimal_metrics_5['sharpe']:.3f}",
        f"{optimal_metrics_5['sortino']:.3f}",
        f"{optimal_metrics_5['max_drawdown']*100:.1f}%",
        f"{optimal_metrics_5['calmar']:.3f}",
        f"{optimal_metrics_5['best_day']*100:.1f}%",
        f"{optimal_metrics_5['worst_day']*100:.1f}%",
        f"{optimal_metrics_5['win_rate']*100:.1f}%",
        f"${optimal_metrics_5['final_value']:,.0f}"
    ]
}

comparison_df = pd.DataFrame(comparison_data)
print("\n" + "="*80)
print("PORTFOLIO COMPARISON SUMMARY")
print("="*80)
print(comparison_df.to_string(index=False))

In [ ]:
# Display optimal allocations
print("\n" + "="*80)
print("OPTIMAL PORTFOLIO ALLOCATIONS")
print("="*80)

print("\n3-Asset Portfolio (BTC/ETH/SOL):")
for asset, weight in zip(assets_3, optimal_weights_3):
    print(f"  {asset}: {weight*100:.0f}%")

print("\n5-Asset Portfolio (BTC/ETH/SOL/BNB/XRP):")
for asset, weight in zip(assets_5, optimal_weights_5):
    print(f"  {asset}: {weight*100:.0f}%")

## 5. Visualizations

In [ ]:
# 5.1 Portfolio Growth Comparison
fig, ax = plt.subplots(figsize=(14, 7))

ax.plot(btc_values.index, btc_values, label='100% BTC', linewidth=2, color='#F7931A')
ax.plot(portfolio_3_values.index, portfolio_3_values, label=f'3-Asset Optimal', linewidth=2, color='#627EEA')
ax.plot(portfolio_5_values.index, portfolio_5_values, label=f'5-Asset Optimal', linewidth=2, color='#14F195')

ax.set_title('Portfolio Growth Comparison (Buy & Hold, No Rebalancing)', fontsize=14, fontweight='bold')
ax.set_xlabel('Date', fontsize=12)
ax.set_ylabel('Portfolio Value ($)', fontsize=12)
ax.legend(loc='upper left', fontsize=11)
ax.set_yscale('log')  # Log scale to better visualize exponential growth
ax.grid(True, alpha=0.3)

# Add annotations for final values
for pv, name, color in [(btc_values, 'BTC', '#F7931A'), 
                         (portfolio_3_values, '3-Asset', '#627EEA'),
                         (portfolio_5_values, '5-Asset', '#14F195')]:
    final_val = pv.iloc[-1]
    ax.annotate(f'${final_val:,.0f}', xy=(pv.index[-1], final_val),
                xytext=(10, 0), textcoords='offset points', fontsize=10, color=color)

plt.tight_layout()
plt.show()

In [ ]:
# 5.2 Drawdown Comparison
def calculate_drawdown_series(portfolio_values):
    rolling_max = portfolio_values.expanding().max()
    drawdowns = (portfolio_values - rolling_max) / rolling_max
    return drawdowns

fig, ax = plt.subplots(figsize=(14, 5))

dd_btc = calculate_drawdown_series(btc_values)
dd_3 = calculate_drawdown_series(portfolio_3_values)
dd_5 = calculate_drawdown_series(portfolio_5_values)

ax.fill_between(dd_btc.index, dd_btc * 100, 0, alpha=0.3, label='100% BTC', color='#F7931A')
ax.fill_between(dd_3.index, dd_3 * 100, 0, alpha=0.3, label='3-Asset', color='#627EEA')
ax.fill_between(dd_5.index, dd_5 * 100, 0, alpha=0.3, label='5-Asset', color='#14F195')

ax.set_title('Drawdown Comparison', fontsize=14, fontweight='bold')
ax.set_xlabel('Date', fontsize=12)
ax.set_ylabel('Drawdown (%)', fontsize=12)
ax.legend(loc='lower left', fontsize=11)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# 5.3 Risk-Return Scatter Plot (Efficient Frontier Visualization)
fig, ax = plt.subplots(figsize=(10, 8))

# Plot all 3-asset combinations
ax.scatter(results_3['volatility'] * 100, results_3['cagr'] * 100, 
           c=results_3['sharpe'], cmap='RdYlGn', alpha=0.5, s=30, label='3-Asset Combinations')

# Highlight key portfolios
portfolios = [
    (btc_metrics['volatility']*100, btc_metrics['cagr']*100, '100% BTC', '#F7931A', 200),
    (optimal_metrics_3['volatility']*100, optimal_metrics_3['cagr']*100, '3-Asset Optimal', '#627EEA', 200),
    (optimal_metrics_5['volatility']*100, optimal_metrics_5['cagr']*100, '5-Asset Optimal', '#14F195', 200),
]

for vol, ret, name, color, size in portfolios:
    ax.scatter(vol, ret, s=size, color=color, edgecolors='black', linewidths=2, zorder=5)
    ax.annotate(name, (vol, ret), xytext=(10, 10), textcoords='offset points', 
                fontsize=11, fontweight='bold')

ax.set_title('Risk-Return Profile (Efficient Frontier)', fontsize=14, fontweight='bold')
ax.set_xlabel('Volatility (Annualized %)', fontsize=12)
ax.set_ylabel('CAGR (%)', fontsize=12)

cbar = plt.colorbar(ax.collections[0], ax=ax)
cbar.set_label('Sharpe Ratio', fontsize=11)

ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# 5.4 Bar Chart Comparison of Key Metrics
fig, axes = plt.subplots(2, 3, figsize=(15, 10))

metrics_to_plot = [
    ('cagr', 'CAGR (%)', 100),
    ('sharpe', 'Sharpe Ratio', 1),
    ('sortino', 'Sortino Ratio', 1),
    ('volatility', 'Volatility (%)', 100),
    ('max_drawdown', 'Max Drawdown (%)', 100),
    ('calmar', 'Calmar Ratio', 1)
]

portfolios_data = [
    ('100% BTC', btc_metrics, '#F7931A'),
    ('3-Asset', optimal_metrics_3, '#627EEA'),
    ('5-Asset', optimal_metrics_5, '#14F195')
]

for idx, (metric_key, metric_name, multiplier) in enumerate(metrics_to_plot):
    ax = axes[idx // 3, idx % 3]
    
    values = [p[1][metric_key] * multiplier for p in portfolios_data]
    colors = [p[2] for p in portfolios_data]
    labels = [p[0] for p in portfolios_data]
    
    bars = ax.bar(labels, values, color=colors, edgecolor='black', linewidth=1)
    ax.set_title(metric_name, fontsize=12, fontweight='bold')
    ax.grid(True, alpha=0.3, axis='y')
    
    # Add value labels on bars
    for bar, val in zip(bars, values):
        height = bar.get_height()
        ax.annotate(f'{val:.2f}' if multiplier == 1 else f'{val:.1f}%',
                    xy=(bar.get_x() + bar.get_width() / 2, height),
                    xytext=(0, 3), textcoords="offset points",
                    ha='center', va='bottom', fontsize=10)

plt.suptitle('Portfolio Metrics Comparison', fontsize=16, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

In [ ]:
# 5.5 Allocation Pie Charts
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

colors_3 = ['#F7931A', '#627EEA', '#14F195']
colors_5 = ['#F7931A', '#627EEA', '#14F195', '#F0B90B', '#23292F']

# BTC Only
axes[0].pie([100], labels=['BTC'], colors=['#F7931A'], autopct='%1.0f%%',
            startangle=90, textprops={'fontsize': 12})
axes[0].set_title('100% BTC', fontsize=14, fontweight='bold')

# 3-Asset
weights_3_pct = optimal_weights_3 * 100
axes[1].pie(weights_3_pct, labels=assets_3, colors=colors_3, autopct='%1.0f%%',
            startangle=90, textprops={'fontsize': 12})
axes[1].set_title('3-Asset Optimal', fontsize=14, fontweight='bold')

# 5-Asset
weights_5_pct = optimal_weights_5 * 100
# Filter out 0% allocations for cleaner pie chart
non_zero_idx = weights_5_pct > 0
axes[2].pie(weights_5_pct[non_zero_idx], 
            labels=[a for a, nz in zip(assets_5, non_zero_idx) if nz], 
            colors=[c for c, nz in zip(colors_5, non_zero_idx) if nz], 
            autopct='%1.0f%%', startangle=90, textprops={'fontsize': 12})
axes[2].set_title('5-Asset Optimal', fontsize=14, fontweight='bold')

plt.suptitle('Optimal Portfolio Allocations', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.show()

In [ ]:
# 5.6 Rolling Sharpe Ratio (1-Year Window)
def calculate_rolling_sharpe(portfolio_values, window=365):
    daily_returns = portfolio_values.pct_change().dropna()
    rolling_mean = daily_returns.rolling(window=window).mean() * 365
    rolling_std = daily_returns.rolling(window=window).std() * np.sqrt(365)
    rolling_sharpe = (rolling_mean - RISK_FREE_RATE) / rolling_std
    return rolling_sharpe

fig, ax = plt.subplots(figsize=(14, 6))

rs_btc = calculate_rolling_sharpe(btc_values)
rs_3 = calculate_rolling_sharpe(portfolio_3_values)
rs_5 = calculate_rolling_sharpe(portfolio_5_values)

ax.plot(rs_btc.index, rs_btc, label='100% BTC', linewidth=2, color='#F7931A')
ax.plot(rs_3.index, rs_3, label='3-Asset', linewidth=2, color='#627EEA')
ax.plot(rs_5.index, rs_5, label='5-Asset', linewidth=2, color='#14F195')

ax.axhline(y=0, color='gray', linestyle='--', alpha=0.5)
ax.axhline(y=1, color='green', linestyle='--', alpha=0.3, label='Sharpe = 1 (Good)')

ax.set_title('Rolling 1-Year Sharpe Ratio', fontsize=14, fontweight='bold')
ax.set_xlabel('Date', fontsize=12)
ax.set_ylabel('Sharpe Ratio', fontsize=12)
ax.legend(loc='upper right', fontsize=11)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# 5.7 Correlation Heatmap
fig, ax = plt.subplots(figsize=(8, 6))

corr_matrix = returns.corr()
mask = np.triu(np.ones_like(corr_matrix, dtype=bool), k=1)

sns.heatmap(corr_matrix, annot=True, cmap='RdYlGn_r', center=0, 
            fmt='.2f', square=True, linewidths=1, ax=ax,
            cbar_kws={'shrink': 0.8})

ax.set_title('Asset Correlation Matrix (Daily Returns)', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

## 6. Monthly Performance Heatmap

In [ ]:
# Calculate monthly returns for each portfolio
def get_monthly_returns(portfolio_values):
    monthly = portfolio_values.resample('M').last()
    monthly_returns = monthly.pct_change().dropna()
    return monthly_returns

# Create monthly returns dataframe
monthly_btc = get_monthly_returns(btc_values)
monthly_3 = get_monthly_returns(portfolio_3_values)
monthly_5 = get_monthly_returns(portfolio_5_values)

# Create heatmap data
def create_monthly_heatmap_data(monthly_returns, name):
    df = pd.DataFrame({'return': monthly_returns})
    df['year'] = df.index.year
    df['month'] = df.index.month
    pivot = df.pivot(index='year', columns='month', values='return')
    pivot.columns = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 
                     'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'][:len(pivot.columns)]
    return pivot

fig, axes = plt.subplots(3, 1, figsize=(14, 12))

for ax, (monthly_data, title) in zip(axes, [
    (monthly_btc, '100% BTC'),
    (monthly_3, '3-Asset Optimal'),
    (monthly_5, '5-Asset Optimal')
]):
    heatmap_data = create_monthly_heatmap_data(monthly_data, title)
    sns.heatmap(heatmap_data * 100, annot=True, fmt='.1f', cmap='RdYlGn', 
                center=0, ax=ax, cbar_kws={'label': 'Return (%)'})
    ax.set_title(f'Monthly Returns: {title}', fontsize=12, fontweight='bold')
    ax.set_xlabel('')
    ax.set_ylabel('Year')

plt.suptitle('Monthly Performance Heatmaps', fontsize=16, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

## 7. Key Findings & Conclusions

In [ ]:
print("\n" + "="*80)
print("KEY FINDINGS & CONCLUSIONS")
print("="*80)

# Determine winners for each metric
all_metrics = [
    ('100% BTC', btc_metrics),
    ('3-Asset', optimal_metrics_3),
    ('5-Asset', optimal_metrics_5)
]

print("\n1. SHARPE RATIO (Primary Metric - Risk-Adjusted Returns):")
sharpe_sorted = sorted(all_metrics, key=lambda x: x[1]['sharpe'], reverse=True)
for i, (name, metrics) in enumerate(sharpe_sorted, 1):
    emoji = ['[1st]', '[2nd]', '[3rd]'][i-1]
    print(f"   {emoji} {name}: {metrics['sharpe']:.3f}")

print("\n2. TOTAL RETURN (Raw Performance):")
return_sorted = sorted(all_metrics, key=lambda x: x[1]['total_return'], reverse=True)
for i, (name, metrics) in enumerate(return_sorted, 1):
    emoji = ['[1st]', '[2nd]', '[3rd]'][i-1]
    print(f"   {emoji} {name}: {metrics['total_return']*100:.1f}%")

print("\n3. MAX DRAWDOWN (Lowest is Best - Downside Protection):")
dd_sorted = sorted(all_metrics, key=lambda x: x[1]['max_drawdown'], reverse=True)  # Less negative is better
for i, (name, metrics) in enumerate(dd_sorted, 1):
    emoji = ['[1st]', '[2nd]', '[3rd]'][i-1]
    print(f"   {emoji} {name}: {metrics['max_drawdown']*100:.1f}%")

print("\n4. SORTINO RATIO (Downside Risk-Adjusted Returns):")
sortino_sorted = sorted(all_metrics, key=lambda x: x[1]['sortino'], reverse=True)
for i, (name, metrics) in enumerate(sortino_sorted, 1):
    emoji = ['[1st]', '[2nd]', '[3rd]'][i-1]
    print(f"   {emoji} {name}: {metrics['sortino']:.3f}")

# Summary
print("\n" + "-"*80)
print("SUMMARY:")
print("-"*80)

best_sharpe_portfolio = sharpe_sorted[0][0]
best_return_portfolio = return_sorted[0][0]

print(f"\n- Best Risk-Adjusted Returns (Sharpe): {best_sharpe_portfolio}")
print(f"- Best Raw Returns: {best_return_portfolio}")

# Compare to BTC
sharpe_improvement_3 = ((optimal_metrics_3['sharpe'] / btc_metrics['sharpe']) - 1) * 100
sharpe_improvement_5 = ((optimal_metrics_5['sharpe'] / btc_metrics['sharpe']) - 1) * 100

print(f"\n- 3-Asset portfolio Sharpe improvement vs BTC: {sharpe_improvement_3:+.1f}%")
print(f"- 5-Asset portfolio Sharpe improvement vs BTC: {sharpe_improvement_5:+.1f}%")

print("\n" + "="*80)

In [ ]:
# Final Summary Table for Export
print("\nFINAL SUMMARY TABLE (Copy-Paste Ready):")
print("\n" + comparison_df.to_markdown(index=False))

## 8. Sensitivity Analysis: Impact of Rebalancing

In [ ]:
# Test quarterly rebalancing vs no rebalancing
def calculate_portfolio_value_with_rebalancing(weights, prices_df, initial_investment=INITIAL_INVESTMENT, rebalance_freq='Q'):
    """
    Calculate portfolio value with periodic rebalancing.
    rebalance_freq: 'Q' for quarterly, 'A' for annual, None for no rebalancing
    """
    portfolio_values = []
    current_value = initial_investment
    
    # Get rebalancing dates
    if rebalance_freq:
        rebalance_dates = prices_df.resample(rebalance_freq).last().index
    else:
        rebalance_dates = []
    
    # Initialize units
    initial_prices = prices_df.iloc[0]
    units = (initial_investment * np.array(weights)) / initial_prices
    
    for date in prices_df.index:
        # Calculate current value
        current_prices = prices_df.loc[date]
        current_value = (units * current_prices).sum()
        portfolio_values.append(current_value)
        
        # Rebalance if it's a rebalancing date
        if rebalance_freq and date in rebalance_dates:
            units = (current_value * np.array(weights)) / current_prices
    
    return pd.Series(portfolio_values, index=prices_df.index)

# Compare with and without rebalancing
print("Impact of Quarterly Rebalancing on 3-Asset Portfolio:\n")

pv_3_no_rebal = calculate_portfolio_value(optimal_weights_3, prices[assets_3])
pv_3_quarterly = calculate_portfolio_value_with_rebalancing(optimal_weights_3, prices[assets_3], rebalance_freq='Q')

metrics_no_rebal = calculate_metrics(pv_3_no_rebal)
metrics_quarterly = calculate_metrics(pv_3_quarterly)

rebal_comparison = pd.DataFrame({
    'Metric': ['CAGR', 'Sharpe Ratio', 'Max Drawdown', 'Final Value'],
    'No Rebalancing': [
        f"{metrics_no_rebal['cagr']*100:.1f}%",
        f"{metrics_no_rebal['sharpe']:.3f}",
        f"{metrics_no_rebal['max_drawdown']*100:.1f}%",
        f"${metrics_no_rebal['final_value']:,.0f}"
    ],
    'Quarterly Rebalancing': [
        f"{metrics_quarterly['cagr']*100:.1f}%",
        f"{metrics_quarterly['sharpe']:.3f}",
        f"{metrics_quarterly['max_drawdown']*100:.1f}%",
        f"${metrics_quarterly['final_value']:,.0f}"
    ]
})

print(rebal_comparison.to_string(index=False))

## 9. Data Export

In [ ]:
# Export results to CSV for further analysis or dashboard integration
export_data = pd.DataFrame({
    'date': prices.index,
    'btc_only': btc_values.values,
    'portfolio_3_asset': portfolio_3_values.values,
    'portfolio_5_asset': portfolio_5_values.values
})

export_data.to_csv('portfolio_backtest_results.csv', index=False)
print("Results exported to 'portfolio_backtest_results.csv'")

# Export optimal weights
weights_export = {
    'portfolio': ['3-Asset Optimal', '5-Asset Optimal'],
    'BTC': [optimal_weights_3[0], optimal_weights_5[0]],
    'ETH': [optimal_weights_3[1], optimal_weights_5[1]],
    'SOL': [optimal_weights_3[2], optimal_weights_5[2]],
    'BNB': [0, optimal_weights_5[3]],
    'XRP': [0, optimal_weights_5[4]],
    'Sharpe': [optimal_metrics_3['sharpe'], optimal_metrics_5['sharpe']],
    'CAGR': [optimal_metrics_3['cagr'], optimal_metrics_5['cagr']]
}
pd.DataFrame(weights_export).to_csv('optimal_weights.csv', index=False)
print("Optimal weights exported to 'optimal_weights.csv'")

---

## Disclaimer

This analysis is for educational and informational purposes only. Past performance does not guarantee future results. Cryptocurrency investments carry significant risk and may result in substantial losses. The optimal allocations shown are based on historical data and may not be optimal going forward. Always conduct your own research and consider consulting a financial advisor before making investment decisions.